# Background

So with the documents indexed into vector database, we can now assemble an application logic. Application logic will retrieve a document from vector store based on the user query. This question along with the document, added to provide context, will be used to assemble a prompt which will then be sent to LLM to generate answer

Here we will first build the application logic step by step and then will encapsulate the logic into Class. We will then register this class as a model into MLFlow where we will track the model development and deployment lifecycle.

In [0]:
%pip install openai==0.27.8 langchain==0.0.251 tiktoken==0.4.0 pdfminer-six==20221105 watchdog==3.0.0 PyMuPDF==1.23.3 faiss-cpu==1.7.4 pysqlite-binary Flask nemoguardrails==0.5.0

INFO:py4j.clientserver:Received command c on object id p0


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)


INFO:py4j.clientserver:Received command c on object id p0


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.0.3
    Uninstalling Jinja2-3.0.3:
      Successfully uninstalled Jinja2-3.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.4.12 requires prometheus-client, which is not installed.
databricks-feature-store 0.14.1 requires pyspark<4,>=3.1.2, which is not installed.


INFO:py4j.clientserver:Received command c on object id p0


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


INFO:py4j.clientserver:Received command c on object id p0


In [0]:
%pip install jinja2==3.0.3

INFO:py4j.clientserver:Received command c on object id p1


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


INFO:py4j.clientserver:Received command c on object id p0


  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.4.12 requires prometheus-client, which is not installed.
databricks-feature-store 0.14.1 requires pyspark<4,>=3.1.2, which is not installed.
nemoguardrails 0.5.0 requires Jinja2==3.1.2, but you have jinja2 3.0.3 which is incompatible.


INFO:py4j.clientserver:Received command c on object id p0


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


INFO:py4j.clientserver:Received command c on object id p0


In [0]:
dbutils.library.restartPython()

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
%run "./utils/config"

In [0]:
import asyncio
import logging
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.vectorstores.faiss import FAISS
import mlflow
import pandas as pd
import time
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

logging.basicConfig(level=logging.INFO)

## Custom Constitutional AI Chain
Currently Langchain's Constitutional-AI only supports LLMChains, hence, we have added the support for all chains. 

In [0]:
"""Chain for applying constitutional principles to the outputs of another chain."""
from typing import Any, Dict, List, Optional

from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.base import Chain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
from langchain.chains.constitutional_ai.principles import PRINCIPLES
from langchain.chains.constitutional_ai.prompts import CRITIQUE_PROMPT, REVISION_PROMPT
from langchain.chains.llm import LLMChain
from langchain.schema import BasePromptTemplate
from langchain.schema.language_model import BaseLanguageModel


class ConstitutionalChain(Chain):
    """Chain for applying constitutional principles.

    Example:
        .. code-block:: python

            from langchain.llms import OpenAI
            from langchain.chains import LLMChain, ConstitutionalChain
            from langchain.chains.constitutional_ai.models \
                import ConstitutionalPrinciple

            llm = OpenAI()

            qa_prompt = PromptTemplate(
                template="Q: {question} A:",
                input_variables=["question"],
            )
            qa_chain = LLMChain(llm=llm, prompt=qa_prompt)

            constitutional_chain = ConstitutionalChain.from_llm(
                llm=llm,
                chain=qa_chain,
                constitutional_principles=[
                    ConstitutionalPrinciple(
                        critique_request="Tell if this answer is good.",
                        revision_request="Give a better answer.",
                    )
                ],
            )

            constitutional_chain.run(question="What is the meaning of life?")
    """

    chain: Chain
    constitutional_principles: List[ConstitutionalPrinciple]
    critique_chain: Chain
    revision_chain: Chain
    return_intermediate_steps: bool = False

    @classmethod
    def get_principles(
        cls, names: Optional[List[str]] = None
    ) -> List[ConstitutionalPrinciple]:
        if names is None:
            return list(PRINCIPLES.values())
        else:
            return [PRINCIPLES[name] for name in names]

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        chain: LLMChain,
        critique_prompt: BasePromptTemplate = CRITIQUE_PROMPT,
        revision_prompt: BasePromptTemplate = REVISION_PROMPT,
        **kwargs: Any,
    ) -> "ConstitutionalChain":
        """Create a chain from an LLM."""
        critique_chain = LLMChain(llm=llm, prompt=critique_prompt)
        revision_chain = LLMChain(llm=llm, prompt=revision_prompt)
        return cls(
            chain=chain,
            critique_chain=critique_chain,
            revision_chain=revision_chain,
            **kwargs,
        )

    @property
    def input_keys(self) -> List[str]:
        """Input keys."""
        return self.chain.input_keys

    @property
    def output_keys(self) -> List[str]:
        """Output keys."""
        if self.return_intermediate_steps:
            return ["output", "critiques_and_revisions", "initial_output"]
        return ["output"]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        response = self.chain.run(
            **inputs,
            callbacks=_run_manager.get_child("original"),
        )
        initial_response = response
        input_prompt = self.chain.prompt.format(**inputs)

        _run_manager.on_text(
            text="Initial response: " + response + "\n\n",
            verbose=self.verbose,
            color="yellow",
        )
        critiques_and_revisions = []
        for constitutional_principle in self.constitutional_principles:
            # Do critique

            raw_critique = self.critique_chain.run(
                input_prompt=input_prompt,
                output_from_model=response,
                critique_request=constitutional_principle.critique_request,
                callbacks=_run_manager.get_child("critique"),
            )
            critique = self._parse_critique(
                output_string=raw_critique,
            ).strip()

            # if the critique contains "No critique needed", then we're done
            # in this case, initial_output is the same as output,
            # but we'll keep it for consistency
            if "no critique needed" in critique.lower():
                critiques_and_revisions.append((critique, ""))
                continue

            # Do revision

            revision = self.revision_chain.run(
                input_prompt=input_prompt,
                output_from_model=response,
                critique_request=constitutional_principle.critique_request,
                critique=critique,
                revision_request=constitutional_principle.revision_request,
                callbacks=_run_manager.get_child("revision"),
            ).strip()
            response = revision
            critiques_and_revisions.append((critique, revision))

            _run_manager.on_text(
                text=f"Applying {constitutional_principle.name}..." + "\n\n",
                verbose=self.verbose,
                color="green",
            )

            _run_manager.on_text(
                text="Critique: " + critique + "\n\n",
                verbose=self.verbose,
                color="blue",
            )

            _run_manager.on_text(
                text="Updated response: " + revision + "\n\n",
                verbose=self.verbose,
                color="yellow",
            )

        final_output: Dict[str, Any] = {"output": response}
        if self.return_intermediate_steps:
            final_output["initial_output"] = initial_response
            final_output["critiques_and_revisions"] = critiques_and_revisions
        return final_output

    @staticmethod
    def _parse_critique(output_string: str) -> str:
        if "Revision request:" not in output_string:
            return output_string
        output_string = output_string.split("Revision request:")[0]
        if "\n\n" in output_string:
            output_string = output_string.split("\n\n")[0]
        return output_string


INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Received command c on object id p0


In [0]:
%run "./utils/constitutional_ai_chain"

# Defining consitutional principles

###Check if offensive

In [0]:
critique_request='Discuss whether the assistant said anything that may be interpreted as offensive or harmful. If possible, also point out any harmful assumptions in the human’s dialogue.'

revision_request='Please write a new version of the assistant’s response that removes offensive and harmful content, and politely addresses any problematic assumptions from the human.'

offensive = ConstitutionalPrinciple(critique_request=critique_request, revision_request=revision_request, name='offensive')

INFO:py4j.clientserver:Received command c on object id p1


INFO:py4j.clientserver:Received command c on object id p1


###Check if its aginat the law

In [0]:
critique_request='Read the assistant’s last response carefully and identify anything it said that provides advice or assistance in potentially criminal activities, such as violence, theft, hacking, robbery, or any other illegal activity. Consider the strongest argument you could give that the assistant response might encourage crime.'

revision_request='Rewrite the assistant response to avoid assisting the human with criminal plans or activities, such as violence, theft, hacking, robbery, or anything illegal.'

criminal = ConstitutionalPrinciple(critique_request=critique_request, revision_request=revision_request, name='criminal')

INFO:py4j.clientserver:Received command c on object id p1


###Check if harmfull

In [0]:
critique_request='Critique the assistants last response and identify anything it said that was harmful, unethical, dangerous, or illegal.'
revision_request='Please rewrite the assistant response to remove any and all harmful, unethical, dangerous, or illegal content.'
harmful = ConstitutionalPrinciple(critique_request=critique_request, revision_request=revision_request, name='harmful7')

INFO:py4j.clientserver:Received command c on object id p1


###Check if ethical and fair

In [0]:
critique_request="The model should only talk about ethical and legal and fair things."
revision_request="Rewrite the model's output to be ethical, legal and fair and should not be harmfull to any living creature."

ethical_principle = ConstitutionalPrinciple(critique_request=critique_request, revision_request=revision_request, name="Ethical Principle")

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
principles = {'offensive': offensive, 'criminal': criminal, 'harmful': harmful, 'ethical_principle': ethical_principle}

# Build question-answer system step-by-step

In [0]:
question = "can you tell me some policies related to personal data protection?"

INFO:py4j.clientserver:Received command c on object id p1


We will retrieve the document chunks that are relevant to a given question from a vector store.

In [0]:
# open vector store to access embeddings
embeddings = OpenAIEmbeddings(model=config['openai_embedding_model'])
vector_store = FAISS.load_local(embeddings=embeddings, folder_path=config['vector_store_path'])

# configure document retrieval 
n_documents = 5 # number of documents to retrieve 
retriever = vector_store.as_retriever(search_kwargs={'k': n_documents}) # configure retrieval mechanism

# get relevant documents
docs = retriever.get_relevant_documents(question)
for doc in docs:
  print(doc,'\n') 

INFO:py4j.clientserver:Received command c on object id p1
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:py4j.clientserver:Received command c on object id p0


page_content='Government Personal Data Protection Policies   |   9\n \nGeneral rules with Respect to \nProtection of Personal Data\n01\nAn agency shall ensure that up-to-date policies and processes that \nadhere to all the provisions in this policy are implemented within the \nagency and when transferring data to any other organisation..02\nAn agency shall implement processes to receive and address, within a \nreasonable time, enquiries or feedback about the agency’s policies and \nprocesses relating to the processing of personal data.' metadata={} 

page_content='Government Personal Data Protection Policies   |   18\n \n \n \n \nAccess to and  \nCorrection of Personal Data\nAccess to Personal Data\n27\nSubject to paragraphs 28, 29 and 31, on request of an individual, an \nagency shall, as soon as reasonably possible, provide the individual  \nwith: — \n(a) personal data about the individual that the individual has earlier \nprovided to the agency; and \n  \n(b) information about the w

INFO:py4j.clientserver:Received command c on object id p0



Allright, now that retrieval from vector store is working as expected, we will build prompt that we will send it to LLM.  Here the prompt is defined using `ChatPromptTemplate` class. This prompt along with the object of LLM are encapsulated in `LLMChain` object.

- System message prompt shown here provides instruction to the model about how we want it to respond.
- The human message template provides the details about the user generated request.

#Create Chain

In [0]:
def get_rails():
    global config
    import nemoguardrails

    rails_config = nemoguardrails.RailsConfig.from_content(
        colang_content=config["rails_colang_config"], 
        yaml_content=config["rails_yaml_config"])
    
    rails_config.config_path=config["rails_config_path"]
    rails = nemoguardrails.LLMRails(rails_config)
    return rails
    
rails = get_rails()

INFO:py4j.clientserver:Received command c on object id p0
INFO:nemoguardrails.actions.action_dispatcher:Initializing action dispatcher
INFO:nemoguardrails.actions.action_dispatcher:Adding wolfram_alpha_request to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding retrieve_relevant_chunks to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding check_jailbreak to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation_v2 to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation to actions
INFO:nemoguardrails.actions.action_dispatcher:Added summarize_document to actions
INFO:nemoguardrails.actions.action_dispatcher:Registered Actions: {'wolfram alpha request': <function wolfram_alpha_request at 0x7f8b3f5f3910>, 'retrieve_relevant_chunks': <function retrieve_relevant_chunks at 0x7f8b3f5f3b50>, 'check_jailbreak': <function check_jailbreak at 0x7f8b3f5f3eb0>, 'output_moderation_v2': <function output_moderation_v2 at 0x7f8b3f5f3

In [0]:
prompt_template = config['system_message_template']
qa_prompt = PromptTemplate.from_template(prompt_template)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(rails.llm, 
                                                vector_store.as_retriever(),
                                                memory=memory, 
                                                condense_question_prompt=qa_prompt)

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
constitutional_chain = ConstitutionalChain.from_llm(
    llm=rails.llm,
    chain=qa_chain,
    constitutional_principles=list(principles.values()),
)

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
rails.register_action(constitutional_chain, name="qa_chain")

INFO:py4j.clientserver:Received command c on object id p1



We will now iterate over the documents from highest to lowest relevancce and try to generate response using chain. As soon as we find a valid response we will break the loop.

In [0]:
# for each provided document
for doc in docs:

  # get document text
  text = doc.page_content

  # generate a response
  output = rails.generate(
            messages=[{"role": "user", "content": question, 'context': text}])
  print(output)

INFO:py4j.clientserver:Received command c on object id p1
INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'UtteranceUserActionFinished', 'final_transcript': 'can you tell me some policies related to personal data protection?'}
INFO:nemoguardrails.flows.runtime:Event :: UtteranceUserActionFinished {'final_transcript': 'can you tell me some policies related to personal data protection?'}
INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'StartInternalSystemAction', 'uid': '3929479d-186d-48ca-ae1f-f1c29e3a2fc3', 'event_created_at': '2023-09-16T10:04:07.894645+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': 'a826bf44-c45e-4422-bbf0-7c7c7bbbe87d', 'is_system_action': True}
INFO:nemoguardrails.flows.runtime:Event :: StartInternalSystemAction {'uid': '3929479d-186d-48ca-ae1f-f1c29e3a2fc3', 'event_created_at': '2023-09-16T10:04:07.894645+00:00', 'source_uid': 'NeMoGuardrails', '

{'role': 'assistant', 'content': 'Yes, I can provide you with information about government personal data protection policies. These policies are designed to protect the privacy of individuals and ensure that their personal data is secure.'}


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2537 request_id=967f1e2400ba0e8fa96158d63c9f53a6 response_code=200
INFO:nemoguardrails.logging.callbacks:Completion ::   ask about policies
bot respond about policies
  "Yes, there are several policies related to personal data protection. The General Data Protection Regulation (GDPR) is a set of regulations that protect the privacy and security of personal data. The California Consumer Privacy Act (CCPA) is another set of regulations that protect the privacy of California residents. Additionally, the Health Insurance Portability and Accountability Act (HIPAA) is a set of regulations that protect the privacy of health information."
INFO:nemoguardrails.logging.callbacks:Output Stats :: {'token_usage': {'completion_tokens': 98, 'prompt_tokens': 355, 'total_tokens': 453}, 'model

{'role': 'assistant', 'content': 'Yes, I can provide you with information about government personal data protection policies. These policies are designed to protect the privacy of individuals and ensure that their personal data is secure.'}


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2694 request_id=c8080fce36016d8307f2e74e2950db3e response_code=200
INFO:nemoguardrails.logging.callbacks:Completion ::   ask about policies
bot respond about policies
  "Yes, there are several policies related to personal data protection. The General Data Protection Regulation (GDPR) is a set of regulations that protect the privacy and security of personal data. The California Consumer Privacy Act (CCPA) is another set of regulations that protect the privacy of California residents. Additionally, the Health Insurance Portability and Accountability Act (HIPAA) is a set of regulations that protect the privacy of health information."
INFO:nemoguardrails.logging.callbacks:Output Stats :: {'token_usage': {'completion_token

{'role': 'assistant', 'content': 'Yes, I can provide you with information about government personal data protection policies. These policies are designed to protect the privacy of individuals and ensure that their personal data is secure.'}


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1709 request_id=cededb2d86b01722223627d8eed4e56f response_code=200
INFO:nemoguardrails.logging.callbacks:Completion ::   ask about policies
bot respond about policies
  "Yes, there are several policies related to personal data protection. The General Data Protection Regulation (GDPR) is a set of regulations that protect the privacy and security of personal data. The California Consumer Privacy Act (CCPA) is another set of regulations that protect the privacy of California residents. Additionally, the Health Insurance Portability and Accountability Act (HIPAA) is a set of regulations that protect the privacy of health information."
INFO:nemoguardrails.logging.callbacks:Output Stats :: {'token_usage': {'completion_tokens': 98, 'prompt_tokens': 355, 'total_tokens': 453}, 'model

{'role': 'assistant', 'content': 'Yes, I can provide you with information about government personal data protection policies. These policies are designed to protect the privacy of individuals and ensure that their personal data is secure.'}


INFO:nemoguardrails.logging.callbacks:Invocation Params :: {'model_name': 'text-davinci-003', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}, '_type': 'openai', 'stop': None}
INFO:nemoguardrails.logging.callbacks:Prompt :: """
You are an intelligent and excellent at answering questions about government policies.
I will ask questions from the documents and you'll help me try finding the answers from it.
The bot is factual and concise. Give the answer using best of your knowledge, say you dont know unable able to answer.

"""

# This is how a conversation between a user and the bot can go:
user "Hello there!"
  express greeting
bot express greeting
  "Hello! How can I assist you today?"
user "What can you do for me?"
  ask about capabilities
bot respond about capabilities
  "I am an AI assistant that helps answer government policies related questions."
user "What's 2+2?"
  ask general que

{'role': 'assistant', 'content': 'Yes, I can provide you with information about government personal data protection policies. These policies are designed to protect the privacy of individuals and ensure that their personal data is secure.'}



# Assemble an encapsulate application for deployment

In [0]:
class QABot():

  def __init__(self, retriever, system_message_template: str, human_message_template: str, embedding_model_name: str, colang_config: str, yaml_config: str, rails_config_path: str):
    tries=0
    while(tries < 3):
        try:
            self.prompt_template = prompt_template
            self.retriever = retriever
            self.rails = self._get_rails(colang_config, yaml_config, rails_config_path)
            # open vector store to access embeddings
            embeddings = OpenAIEmbeddings(model=embedding_model_name)

            # combine instructions into a single prompt
            chat_prompt = ChatPromptTemplate.from_messages([
                                                            # define system-level instructions
                                                            SystemMessagePromptTemplate.from_template(system_message_template),
                                                            # define human-driven instructions 
                                                            HumanMessagePromptTemplate.from_template(human_message_template)
                                                            ])

            memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

            self.qa_chain = ConversationalRetrievalChain.from_llm(
                                                                    self.rails.llm, 
                                                                    self.retriever,
                                                                    memory=memory, 
                                                                    combine_docs_chain_kwargs={"prompt": chat_prompt})
            self.rails.register_action(self.qa_chain, name="qa_chain")
            break
        except Exception as e:
            if "There is no current event loop in thread" in str(e):
                loop = asyncio.new_event_loop()
                asyncio.set_event_loop(loop)
            if tries == 2:
                raise e
        
        tries = tries + 1


  def _get_rails(self, colang_config, yaml_config, rails_config_path):
        import nemoguardrails
        rails_config = nemoguardrails.RailsConfig.from_content(
                colang_content=colang_config, 
                yaml_content=yaml_config)
        rails_config.config_path=rails_config_path
        rails = nemoguardrails.LLMRails(rails_config)
        return rails


  def _is_good_answer(self, answer):

    ''' check if answer is a valid '''

    result = True # default response

    if answer is None: # bad answer if answer is none
      results = False
    
    return result


  def _get_answer(self, context, question, timeout_sec=60):

    '''' get answer from llm with timeout handling '''

    # default result
    result = None

    # define end time
    end_time = time.time() + timeout_sec

    # try timeout
    while time.time() < end_time:

      # attempt to get a response
      try: 
        docs = self.retriever.get_relevant_documents(question)

        result = self.rails.generate(
            messages=[{"role": "user", "content": question, 'context': doc}])
        result = result["content"]
        break # if successful response, stop looping

      # if rate limit error...
      except openai.error.RateLimitError as rate_limit_error:
        if time.time() < end_time: # if time permits, sleep
          time.sleep(2)
          continue
        else: # otherwise, raiser the exception
          raise rate_limit_error

      # if other error, raise it
      except Exception as e:
        print(f'OOPSY LLM QA Chain encountered unexpected error: {e}')
        raise e

    return result


  def get_answer(self, question):
    ''' get answer to provided question '''

    # default result
    result = {'question':None, 'answer':None}

    # get relevant documents
    docs = self.retriever.get_relevant_documents(question)

    # for each doc ...
    for doc in docs:

      # get key elements for doc
      text = doc.page_content

      # get an answer from llm
      answer = self._get_answer(text, question)
 
      # assemble results if not no_answer
      if self._is_good_answer(answer):
        result['answer'] = answer
        result['question'] = question
        break # stop looping if good answer
      
    return result

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
# instantiate bot object
qabot = QABot(retriever, 
              config['system_message_template'], 
              config['human_message_template'], 
              config['openai_embedding_model'], 
              config["rails_colang_config"], 
              config["rails_yaml_config"], 
              config["rails_config_path"])

# get response to question
qabot.get_answer(question)

INFO:py4j.clientserver:Received command c on object id p1
INFO:nemoguardrails.actions.action_dispatcher:Initializing action dispatcher
INFO:nemoguardrails.actions.action_dispatcher:Adding wolfram_alpha_request to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding retrieve_relevant_chunks to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding check_jailbreak to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation_v2 to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation to actions
INFO:nemoguardrails.actions.action_dispatcher:Added summarize_document to actions
INFO:nemoguardrails.actions.action_dispatcher:Registered Actions: {'wolfram alpha request': <function wolfram_alpha_request at 0x7f8b3f4131c0>, 'retrieve_relevant_chunks': <function retrieve_relevant_chunks at 0x7f8b3f412050>, 'check_jailbreak': <function check_jailbreak at 0x7f8b3f4105e0>, 'output_moderation_v2': <function output_moderation_v2 at 0x7f8b3f412

{'question': 'can you tell me some policies related to personal data protection?',
 'answer': 'Yes, I can help you with that. The General Data Protection Regulation (GDPR) is a regulation in EU law on data protection and privacy for all individuals within the European Union. It also addresses the export of personal data outside the EU. The GDPR aims primarily to give control to citizens and residents over their personal data and to simplify the regulatory environment for international business by unifying the regulation within the EU.\nThe above response may have been hallucinated, and should be independently verified.'}


# Load the evaluation data set

In [0]:
# create dataset fromt the raw text data
eval_data = (
  spark
    .table('generated_policies_qa_pair')
    .selectExpr(
      'question',
      'answer'
      )
  )
display(eval_data)

INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0


question,answer
"According to the document, what is the purpose of the Government Instruction Manual (IM) on Infocomm Technology and Smart Systems (ICT&SS) Management?",The purpose of the Government Instruction Manual (IM) on Infocomm Technology and Smart Systems (ICT&SS) Management is to set out the key policies that govern how data security is managed by agencies.
What are the two legal frameworks governing data management in the public and private sectors?,"The Public Sector (Governance) Act (""PSGA"") governs data management in the public sector, while the Personal Data Protection Act (""PDPA"") applies to the private sector."
"According to the document, what are the three key stages in the lifecycle of the Government's Third-Party Management Framework?","The three key stages in the lifecycle of the Government's Third-Party Management Framework are Evaluation and Selection, Contracting and On-boarding, and Service Management and Transition Out."



# Save & Evaluate the model with MLflow

In [0]:
class MLflowQABot(mlflow.pyfunc.PythonModel):

  def __init__(self, retriever, 
               system_message_template: str, 
               human_message_template: str, 
               embedding_model_name: str, 
               colang_config: str, 
               yaml_config: str, 
               rails_config_path: str):
      
    self.qabot = QABot(retriever, 
                       system_message_template, 
                       human_message_template, 
                       embedding_model_name, 
                       colang_config, 
                       yaml_config, 
                       rails_config_path)

  def predict(self, context, inputs):
    questions = list(inputs['question'])

    # return answer
    return [self.qabot.get_answer(q) for q in questions]

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
# instantiate mlflow model
import mlflow
model = MLflowQABot(retriever, 
                    config['system_message_template'], 
                    config['human_message_template'], 
                    config['openai_embedding_model'], 
                    config["rails_colang_config"], 
                    config["rails_yaml_config"], 
                    config["rails_config_path"])
logged_model = None

# persist model to mlflow
mlflow_run = mlflow.start_run(run_name = "policy-qa-with-rails-bot1")

logged_model = mlflow.pyfunc.log_model(
    python_model=model,
    extra_pip_requirements=['openai==0.27.8', 'langchain==0.0.251', 'nemoguardrails==0.5.0', 'tiktoken==0.4.0', 'watchdog==3.0.0', 'PyMuPDF==1.23.3', 'pysqlite-binary', 'faiss-cpu==1.7.4', 'Flask'],
    artifact_path='model',
    registered_model_name=config['registered_model_name']
    )
print(str(logged_model.model_uri))

INFO:py4j.clientserver:Received command c on object id p1
INFO:nemoguardrails.actions.action_dispatcher:Initializing action dispatcher
INFO:nemoguardrails.actions.action_dispatcher:Adding wolfram_alpha_request to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding retrieve_relevant_chunks to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding check_jailbreak to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation_v2 to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation to actions
INFO:nemoguardrails.actions.action_dispatcher:Added summarize_document to actions
INFO:nemoguardrails.actions.action_dispatcher:Registered Actions: {'wolfram alpha request': <function wolfram_alpha_request at 0x7f8b3f413d90>, 'retrieve_relevant_chunks': <function retrieve_relevant_chunks at 0x7f8b3f410670>, 'check_jailbreak': <function check_jailbreak at 0x7f8b3f412440>, 'output_moderation_v2': <function output_moderation_v2 at 0x7f8b3e088

runs:/8d6de7c0478b41af9a4c4695208bade9/model


Created version '36' of model 'llm-qa-government-policy-bot'.


In [0]:
mlflow.evaluate(
    model = logged_model.model_uri,
    model_type = "question-answering",
    data = eval_data
)

INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
2023/09/16 10:05:25 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
INFO:py4j.clientserver:Received command c on object id p0
INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'UtteranceUserActionFinished', 'final_transcript': 'According to the document, what is the purpose of the Government Instruction

In [0]:
mlflow.end_run()

INFO:py4j.clientserver:Received command c on object id p1


In [0]:
results: pd.DataFrame = mlflow.load_table(
    "eval_results_table.json", extra_columns=["run_id"]
)
results_grouped_by_question = results.sort_values(by="question")
print("Evaluation results:")
print(
    results_grouped_by_question[
        ["run_id", "question", "outputs"]
    ]
)

INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Closing down clientserver connection


Evaluation results:
                             run_id  ...                                            outputs
4  1420edc58db540f4a44356343353b688  ...  {'question': 'According to the document, what ...
7  7f08d4eecda446d49304e80eb775b3a0  ...  {'question': 'According to the document, what ...
2  8d6de7c0478b41af9a4c4695208bade9  ...  {'question': 'According to the document, what ...
0  8d6de7c0478b41af9a4c4695208bade9  ...  {'question': 'According to the document, what ...
1  8d6de7c0478b41af9a4c4695208bade9  ...  {'question': 'What are the two legal framework...
5  1420edc58db540f4a44356343353b688  ...  {'question': 'What are the two legal framework...
8  7f08d4eecda446d49304e80eb775b3a0  ...  {'question': 'What are the two legal framework...
3  1420edc58db540f4a44356343353b688  ...  {'question': 'What is the purpose of the Gover...
6  7f08d4eecda446d49304e80eb775b3a0  ...  {'question': 'What is the purpose of the Gover...

[9 rows x 3 columns]


In [0]:
# connect to mlflow 
client = mlflow.MlflowClient()

# identify latest model version
latest_version = client.get_latest_versions(config['registered_model_name'], stages=['None'])[0].version

# move model into production
client.transition_model_version_stage(
    name=config['registered_model_name'],
    version=latest_version,
    stage='Production',
    archive_existing_versions=True
)

INFO:py4j.clientserver:Received command c on object id p1


<ModelVersion: aliases=[], creation_timestamp=1694858716532, current_stage='Production', description='', last_updated_timestamp=1694858754564, name='llm-qa-government-policy-bot', run_id='8d6de7c0478b41af9a4c4695208bade9', run_link='', source='dbfs:/databricks/mlflow-tracking/490109738301956/8d6de7c0478b41af9a4c4695208bade9/artifacts/model', status='READY', status_message='', tags={}, user_id='2871141100034611', version='36'>

In [0]:
# retrieve model from mlflow
model = mlflow.pyfunc.load_model(f"models:/{config['registered_model_name']}/Production")

# assemble question input
queries = pd.DataFrame({'question':[
  "List the policies related to personal data protection",
  "How to read data with Delta Sharing?"
]})

# get a response
model.predict(queries)

INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'UtteranceUserActionFinished', 'final_transcript': 'List the policies related to personal data protection'}
INFO:nemoguardrails.flows.runtime:Event :: UtteranceUserActionFinished {'final_transcript': 'List the policies related to personal data protection'}
INFO:nemoguardrails.flows.runtime:Processing event: {'type': 'StartInternalSystemAction', 'uid': 'd3a6ed62-d777-4e44-aaa8-1d005bca6eeb', 'event_created_at': '2023-09-16T10:06:44.851702+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'a

[{'question': 'List the policies related to personal data protection',
  'answer': 'The policies related to personal data protection include the General Data Protection Regulation (GDPR), the California Consumer Privacy Act (CCPA), and the Personal Data Protection Bill (PDPB).\nThe above response may have been hallucinated, and should be independently verified.'},
 {'question': 'How to read data with Delta Sharing?',
  'answer': 'Delta Sharing is a data sharing protocol that allows for secure and efficient data sharing between two or more parties. It is based on the principles of data encryption, authentication, and authorization. To read data with Delta Sharing, you must first authenticate the data source, then encrypt the data, and finally authorize the data to be shared.'}]

In [0]:
print("Completed!")

INFO:py4j.clientserver:Received command c on object id p1


Completed!
